# Remove files from S3
- This notebook is to be used to remove files from s3 and can be used to mass remove files from a folder
- Use this notebook when there are a lot of files to be removed, saving time vs using WinSCP

In [1]:
import os
import boto3
import pandas as pd

from tqdm import tqdm
from dotenv import load_dotenv

load_dotenv()

True

## Connect do S3

In [2]:
s3_client = boto3.client(
    's3',
    aws_access_key_id=os.environ.get('AWS_ACCESS_KEY_ID'),
    aws_secret_access_key=os.environ.get('AWS_SECRET_ACCESS_KEY')
)

BUCKET = 'ddpawsfs'
KEY = 'workspace/mkt-cli-solutions/namos/'

# check if bucket exists
list_of_buckets = [r['Name'] for r in s3_client.list_buckets()['Buckets']]
if not BUCKET in list_of_buckets: raise Exception()

## check files to delete

In [5]:
paginator = s3_client.get_paginator('list_objects_v2')
pages = paginator.paginate(Bucket=BUCKET, Prefix=KEY+'ixcard')

objects_to_delete = []
for page in tqdm(pages):
    for obj in page['Contents']:
        objects_to_delete.append(obj['Key'])

counter_uploaded = pd.DataFrame(objects_to_delete)
print(counter_uploaded.shape)
counter_uploaded.head()

55it [00:14,  3.92it/s]

(54002, 1)


,0
0,workspace/mkt-cli-solutions/namos/ixcard/_SUCCESS
1,workspace/mkt-cli-solutions/namos/ixcard/year=...
2,workspace/mkt-cli-solutions/namos/ixcard/year=...
3,workspace/mkt-cli-solutions/namos/ixcard/year=...
4,workspace/mkt-cli-solutions/namos/ixcard/year=...


## single file
delete a single file frm S3

## multi files using pagination
- when there are more than 1000 files to be deleted, using boto will require pagination because boto only lists 1000 files at the time
- boto3 delete function will also delete 1000 files each time

In [6]:
paginator = s3_client.get_paginator('list_objects_v2')
pages = paginator.paginate(Bucket=BUCKET, Prefix=KEY+'ixcard')

objects_to_delete = {'Objects': []}

for obj in tqdm(pages.search('Contents')):
    objects_to_delete['Objects'].append({'Key': obj['Key']})

    # flush once aws limit reached
    if len(objects_to_delete['Objects']) >= 1000:
        s3_client.delete_objects(Bucket=BUCKET, Delete=objects_to_delete)
        objects_to_delete = {'Objects': []}
        
if len(objects_to_delete['Objects']) < 1000:
    s3_client.delete_objects(Bucket=BUCKET, Delete=objects_to_delete)

54002it [01:15, 716.93it/s]
